In [2]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np
from glob import glob

In [8]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(3*200*180,2000 ),
            nn.Tanh(),
            nn.Linear(2000, 1000),
            nn.Tanh(),
            nn.Linear(1000, 200),
            nn.Tanh(),
            nn.Linear(200, 100),   
        )
        self.decoder = nn.Sequential(
            nn.Linear(100, 200),
            nn.Tanh(),
            nn.Linear(200, 1000),
            nn.Tanh(),
            nn.Linear(1000, 2000),
            nn.Tanh(),
            nn.Linear(2000, 3*200*180),
            nn.Sigmoid(),       
        )

    def forward(self, x):
        encoded = self.encoder(x)
        print("-"*100)
        print(len(encoded[0]))
        decoded = self.decoder(encoded)
        return encoded, decoded




In [3]:

root_dir = 'faces94' 
files = glob(f"{root_dir}/**/**/**/*.jpg")

c2i = {
    "female": 0,
    "male":1,
    "malestaff": 2 
}

c_files = files[:50]

In [4]:


transform_loader = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])


In [5]:
from PIL import Image
from sklearn.model_selection import train_test_split
all_imgs = torch.stack([transform_loader(Image.open(x)) for x in c_files])
all_label = torch.tensor([c2i[x.split('\\')[-3]] for x in c_files])

train_idx, test_idx = train_test_split(range(len(all_imgs)), test_size=0.2, random_state=102)

train_img = all_imgs[train_idx]
train_label = all_label[train_idx]

test_img = all_imgs[test_idx]
test_label = all_label[test_idx]


EPOCH = 10
BATCH_SIZE = 64
LR = 0.005         
DOWNLOAD_MNIST = True
N_TEST_IMG = 5

In [ ]:

autoencoder = AutoEncoder()

optimizer = torch.optim.Adam(autoencoder.parameters(), lr=LR)
loss_func = nn.MSELoss()
for epoch in range(0,5):

    for step,x in enumerate(train_img):
        b_x = x.view(-1, 3*200*180)   
        b_y = x.view(-1, 3*200*180)   
        
        encoded, decoded = autoencoder(b_x)
        print(encoded)
        np.savetxt("wei.csv",encoded.detach().cpu().numpy()[0],delimiter=",")
        
        loss = loss_func(decoded, b_y)      
        optimizer.zero_grad()               
        loss.backward()
        optimizer.step()                    

        if step % 100 == 0:
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy())
        break
        


----------------------------------------------------------------------------------------------------
100
tensor([[ 2.0602e-02, -7.9836e-02,  1.8840e-02,  1.0719e-01, -1.7152e-02,
         -8.3083e-02,  3.2845e-02, -1.3921e-01,  3.3041e-02,  6.9663e-02,
         -3.0524e-02,  2.8641e-02,  8.7731e-02, -5.3914e-02,  3.1062e-02,
         -7.9491e-03, -6.2171e-02,  7.5188e-02, -7.5636e-03, -1.2576e-01,
          8.4847e-02, -6.6230e-02, -3.4097e-02,  1.1698e-01, -1.4058e-02,
          9.2134e-04,  9.3304e-02,  3.0077e-02,  7.0536e-02, -4.3180e-02,
          7.9889e-05,  2.3725e-02, -4.9694e-02, -1.2361e-02, -1.1389e-01,
          4.9921e-03,  3.9231e-02, -5.0697e-02,  3.9146e-03, -3.4713e-02,
          1.1569e-01, -1.6963e-02,  1.9433e-03,  4.0434e-02,  7.8282e-02,
         -8.1527e-03, -5.9038e-02, -6.1727e-03,  6.1871e-02,  7.2813e-02,
          2.1582e-02,  6.8640e-02, -3.3016e-03, -8.3042e-02, -3.9683e-02,
          6.0496e-02, -8.8021e-02,  4.9300e-02,  6.8347e-02, -1.6637e-02,
       